# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv("../data/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
#Aggregate the amount spent for all unique customers
sum_by_customer = orders.groupby(["CustomerID"]).sum().copy()
sum_by_customer.head()

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
12346,61619,541431,2011,1,2,10,74215,1.04,77183.60
12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00
12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24
12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55
12350,1365627,9231629,34187,34,51,272,197,65.30,334.40


In [4]:
#Create dictionary with clients and total amounts
total_amounts = pd.Series(sum_by_customer.amount_spent.values,sum_by_customer.index).to_dict()

#Create a new column in dataframe maping 'CustomerID' with 'total_amounts' dictionary
orders["total_amount"] = orders["CustomerID"].map(total_amounts)
orders.head()

#Assign percentiles for 'Preferred' (top 30%) and 'VIP' (top 10%)
pref_percentile = 0.7
vip_percentile = 0.9

pref_min_amount = round(orders["total_amount"].quantile(pref_percentile), 2)
vip_min_amount = round(orders["total_amount"].quantile(vip_percentile), 2)

print("Customers are 'Preferred' if they spend more than", pref_min_amount)
print("Customers are 'VIP' if they spend more than", vip_min_amount)

Customers are 'Preferred' if they spend more than 5045.61
Customers are 'VIP' if they spend more than 28337.38


In [5]:
#Create new column with customer class. "Regular" by defect
orders["customer_class"] = np.nan

#Fill the classes using a ternary
orders["customer_class"] = np.where(orders["total_amount"] >= vip_min_amount, "VIP", 
                        (np.where(orders["total_amount"] < pref_min_amount, "Regular", 
                        "Preferred")))

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,total_amount,customer_class
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

Provide your solution for Q2 below:

In [6]:
#Remove multiple orders from customers (keep only the first of them)
customers = orders.drop_duplicates(subset="CustomerID", keep='first')

#Clean unnecessary columns
customers_country_class = customers[["Country", "customer_class"]]

customers_country_class.head()

,Country,customer_class
0,United Kingdom,Preferred
9,United Kingdom,Regular
26,France,Preferred
46,United Kingdom,Regular
65,United Kingdom,Regular


In [7]:
#Filter by VIP 'customer_class'
vips_by_country = customers_country_class.loc[customers_country_class["customer_class"] == "VIP"]

#Count the occurrences of each country
vips_by_country = vips_by_country.groupby(["Country"]).count()

vips_by_country

,customer_class
Country,
Australia,1
EIRE,2
Netherlands,1
Sweden,1
United Kingdom,29


In [8]:
#Select the country with most VIPs
most_vips_country = vips_by_country.loc[vips_by_country["customer_class"].idxmax()]

most_vips_country

customer_class    29
Name: United Kingdom, dtype: int64

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [9]:
#Filter by non regular customer class
top_costumers_by_country = customers_country_class.loc[customers_country_class["customer_class"] != "Regular"]

#Count the occurrences of each country
top_costumers_by_country = top_costumers_by_country.groupby(["Country"]).count()

#Select the country with highest number of good clients
top_costumers_by_country = top_costumers_by_country.loc[top_costumers_by_country["customer_class"].idxmax()]

top_costumers_by_country

customer_class    224
Name: United Kingdom, dtype: int64